In [19]:
import pandas as pd
import numpy as np
import os
from PIL import Image


In [20]:
folder = 'ATT'
file_list = [f for f in os.listdir(folder) if f.endswith('.jpg') or f.endswith('.png')]
df = pd.DataFrame(file_list, columns=['filename'])

In [21]:
def PCA(images):
    mean_image = np.mean(images, axis=0)
    centered_images = images - mean_image
    n_samples = centered_images.shape[0]
    covariance_matrix = np.dot(centered_images.T, centered_images) / (n_samples - 1)
    eigenvalues, eigenvectors = np.linalg.eig(covariance_matrix)
    idx = np.argsort(eigenvalues)[::-1]
    sorted_eigenvectors = eigenvectors[:, idx]
    principal_components = sorted_eigenvectors[:, :60]
    reduced_images = np.dot(centered_images, principal_components)
    return reduced_images

In [22]:
def read_image(file_name):
    img_path = os.path.join(folder, file_name)
    img = Image.open(img_path).convert('L')
    img_array = np.array(img)
    return img_array

df['image_array'] = df['filename'].apply(read_image)
df['flatten_array'] = df['image_array'].apply(lambda x: x.flatten())

In [23]:
img_data = df['flatten_array'].tolist()
labels = df['filename'].tolist()

In [24]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
img_data = scaler.fit_transform(img_data)

In [25]:
def KNN(prior_data,prior_labels,input_data):
    distance=[]
    for data in prior_data:
        difference=data-input_data
        distance.append(np.linalg.norm(difference, axis=0))
    tmp=1000000
    index=0
    for i,d in enumerate(distance):
        if d<tmp:
            index=i
            tmp=d
    return prior_labels[index]

In [26]:
def calcu_accuracy(list1, list2):
    total = len(list1)
    matches = 0
    for s1, s2 in zip(list1, list2):
        prefix1 = s1.split('_')[0]
        prefix2 = s2.split('_')[0]
        if prefix1 == prefix2:
            matches += 1
    return matches / total

In [27]:
unit=int(len(img_data)/5)
accurcy=0
for i in range(5):
    output=[]
    for input_data in img_data[unit*i:unit*(i+1)]:
        train_data = np.concatenate((img_data[:unit*i], img_data[unit*(i+1):]), axis=0)
        train_label = np.concatenate((labels[:unit*i], labels[unit*(i+1):]), axis=0)
        output.append(KNN(train_data,train_label,input_data))
    test_label=labels[unit*i:unit*(i+1)]
    accurcy+=calcu_accuracy(test_label,output)
print("average accuracy without PCA:",accurcy/5)

average accuracy without PCA: 0.9199999999999999


In [31]:
img_data = PCA(img_data)

In [32]:
unit=int(len(img_data)/5)
accurcy=0
for i in range(5):
    output=[]
    for input_data in img_data[unit*i:unit*(i+1)]:
        train_data = np.concatenate((img_data[:unit*i], img_data[unit*(i+1):]), axis=0)
        train_label = np.concatenate((labels[:unit*i], labels[unit*(i+1):]), axis=0)
        output.append(KNN(train_data,train_label,input_data))
    test_label=labels[unit*i:unit*(i+1)]
    accurcy+=calcu_accuracy(test_label,output)
print("average accuracy with PCA:",accurcy/5)

average accuracy with PCA: 0.9349999999999999
